<a href="https://colab.research.google.com/github/f901107/OpenAI-Whisper-speech-recognition/blob/main/OpenAI_Whisper_speech_recognition_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#OpenAI 的 Whisper 語音辨識

* 可使用 .txt 檔案放要辨識的清單。
* 可自訂字幕檔要儲存的資料夾。
* 可自訂是否要跳過已辨識過的。
* 改用 yt-dlp 下載影音檔案。


OpenaAI 的 Whisper 是一個自動語音辨識系統，而且有開源，可以在底下的網址中找到:
* https://github.com/openai/whisper

結合 Whisper 和 pytube 或其它類似 yt-dlp 的工具，就可以將 Youtube 上的影片或播放清單擷取聲音、儲存語音檔後，進行語音辨識，並生成字幕檔。

目前在後面程式設定區塊中，語音來源路徑的「url」欄位中，可以填入 Youtube 的影片或影片清單網址；如果是在電腦本機中的影片或聲音檔，則可以上載在左側資料夾中後，在「url」欄位中填入完整的檔名名稱。

接著將其它選項都設定好後，就可以在[程式區塊]中按「執行」的按鈕，開始進行語音辨識了。

Whisper 也可以辨識臺語，不過記得設定時，語言代碼「lang」的欄位要選「Chinese」比較保險，如果用「自動判斷」，有時會被當成非中文，而無法辨識出文字。

程式第一次執行時，因為要安裝及下載自動語音辨識所需要的資料，可能要稍等一下下。

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
#%cd /content/drive

Mounted at /content/drive


In [ ]:
#@title OpenAI Whisper 語音辨識並輸出字幕檔案程式 { vertical-output: true }

#@markdown <b>設定底下的自訂參數後，就可以按左側的執行鈕</b>

#@markdown <b>聲音檔的來源</b>，可以是網址(Youtube影片、撥放清單)，或是上載後的影片、聲音檔檔案名稱。<br />有多個可以放入 .txt 檔案中(例: list.txt)，一行一個網址或是檔案路徑。
url = "" #@param {type:"string"}
#@markdown <b>語音的語言代碼</b>
lang = 'Chinese' #@param ["Chinese", "English", "Japanese", "Korean", "自動判斷"]
#@markdown <b>輸出為哪一種格式</b>（.srt:字幕檔、.txt:純文字檔）
outputFormat = 'txt' #@param ['srt', 'txt']
#@markdown <b>輸出到哪個資料夾</b>(空白表示使用預設值)
output_path = '' #@param {type:"string"}
#@markdown <b>是否覆蓋已存在的辨識結果</b>(沒勾選會跳過已辨識過的)
overwrite_enable = True #@param { type: 'boolean' }
#@markdown <b>使用哪一種辨識模型</b>（small:快/普通，medium:慢/精準, large:很慢/最精準[建議使用]）
modelType = 'large' #@param ["small" , "medium","large"]
#@markdown <b>是否全部辨識完成，立即下載字幕檔</b>
start_downloading_immediately = True #@param { type: 'boolean' }
#@markdown <b>是否即時顯示語音辨識結果</b>
verbose = True #@param { type: 'boolean' }
#markdown <b>是否使用 yt-dlp 下載 Youtube 聲音檔</b><br />(如果較長的聲音檔下載發生問題時再勾選使用)
#YoutubeDL_enable = False #@param { type: 'boolean' }

#@markdown ---
#@markdown <center><p><font color="blue">以下為指令輸出內容</p></center>

# Install + Import + Config
try: import whisper
except:
  print('install whisper ...')
  ! pip -qq install git+https://github.com/openai/whisper.git

try: from yt_dlp import YoutubeDL 
except :
  print('install yt_dlp ...')
  ! pip -qq install yt_dlp

  
try: from slugify import slugify
except:
  print('install python-slugify ...')
  ! pip -qq install python-slugify


import torch
import whisper
from whisper.utils import get_writer
from yt_dlp import YoutubeDL
import re
import os
import urllib.request
from slugify import slugify
import google

audioFile = 'source.mp3' #暫存的語音檔檔名
tempFolder = '.' #暫存的資料夾(工作目錄、下載的影音、剛轉好的文字檔)
title = ''
textFileList = [] #記錄轉好的文字檔檔名清單

#檢查輸出文字檔的資料夾是否存在, 不存在就改用預設值
if re.sub('\s', '', output_path) == '' :
  #如果沒有輸入，就輸出到預設的暫存資料夾
  output_path = tempFolder
else :
  if os.path.exists(output_path) :
    if not os.path.isdir(output_path) :
      print(output_path + ' 不是資料夾哦! 改用預設值: '+tempFolder)
      output_path = tempFolder
  else :
    print('指定的輸出目錄 '+output_path+' 不存在, 改用預設值: '+tempFolder)
    output_path = tempFolder


# GPU or CPU
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

def getYoutubePlaylistInfo_ydl(url) :
  """
  使用 yt_dlp 由 Youtbue 播放清單網址擷取各影片的資訊
  :param url: Youtube 影片播放清單網址
  """  
  ydl_opts = {
      'quiet':True,
  }
  with YoutubeDL(ydl_opts) as ydl: 
    info_dict = ydl.extract_info(url, download=False)
    return info_dict

def getYoutubeTitle_ydl(url) :
  """
  使用 yt_dlp 由 Youtbue 網址擷取影片的標題字
  :param url: Youtube 影片網址
  """  
  ydl_opts = {
      'quiet':True
  }
  with YoutubeDL(ydl_opts) as ydl: 
    info_dict = ydl.extract_info(url, download=False)
    return(info_dict.get('title', None))

def getAudioFromYoutube_ydl(url, output_path, outputFilename):
  """
  使用 yt_dlp 由 Youtbue 網址下載影片的聲音並存檔
  :param url: Youtube 影片網址
  :param output_path: 儲存的資料夾名稱
  :param outputFilename: 聲音檔的檔名
  """
  #filename為去掉副檔名的路徑，以免變成 xxx.mp3.mp3
  filename = os.path.join(output_path, os.path.splitext(outputFilename)[0])
  ydl_opts = {
      'quiet':True,
      'outtmpl':filename,
      'keepvideo':False,
      'format': 'bestaudio/best',
      'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
      #  'preferredquality': '192',
      }],      
      'overwrites':True
  }
  with YoutubeDL(ydl_opts) as ydl: 
    ydl.download(url)

def transcribe(audioFilename, output_path, outputFilename, outputFormat) :
  """
  load whisper model and get transcribe result
  :param audioFilename: audio file to transcribe
  :param output_path: the folder to save result
  :param outputFilename: the filename(without the extension) to save result  
  :param outputFormat: the transcribe result format (thie extension of filename)
  """

  model = whisper.load_model(modelType, device=DEVICE)
  if lang=="自動判斷" :
    print('auto detect language')
    result = model.transcribe(audioFilename, fp16=False, verbose=verbose)    
  else :
    result = model.transcribe(audioFilename, fp16=False, verbose=verbose, language=lang)
  # save result to outputFormat file
  saveToFile(result, output_path, outputFilename, outputFormat)

def saveToFile(result, output_path, filename, fileType='srt') :
  """
  save whisper transcribe result to a file
  :param result: whisper transcribe result
  :param output_path: the folder to save result
  :param filename: the filename(without the extension) to save result  
  :param fileType: the transcribe result format (thie extension of filename)
  """
  file_writer = get_writer(fileType, output_path) #whisper get_writer

  #可標定字的時間截記 word_timestamps 必須傳入 writer_args 參數
  writer_args = {'highlight_words': False, 'max_line_count': None, 'max_line_width': None}
  file_writer(result, filename, writer_args)


#
def getFilenameFromTitle(title) :
  """
  convert title to valid filename
  :param title: text to slugify
  """
  return slugify(title, allow_unicode=True, lowercase=False)

def getOutputTextFilename(filename) :
  fPath = os.path.relpath(output_path)
  return os.path.join(fPath, filename+'.'+outputFormat)

def isOutputTextFileExists(filename) :
  return os.path.exists(getOutputTextFilename(filename))

def removeAudioFile(filename) :
  if os.path.exists(filename) :
    os.remove(filename)

def convertNewLineFormat(filename) :
  if os.path.exists(filename) :
    f = open(filename, 'r+')
    data = f.read()
    data = re.sub('\r', '', data, re.MULTILINE) #remove all \r
    data = re.sub('\n', '\r\n', data, re.MULTILINE) #replace \n to \r\n
    f.seek(0)
    f.write(data)
    f.truncate()
    f.close()

def getAudioAndTranscribe(urlOrFile) :
  filename = audioFile #語音檔檔名(辨識用的暫存檔，用固定的名稱較方便)
  # 分網址或是Colab儲存空間檔案處理
  if re.search('https\:\/\/', urlOrFile) :
    isPlayList = False
    # Youtube 分播放清單跟單一影片處理
    if re.search('youtube\.|youtu\.', urlOrFile) :
      # 建立 Playlist 物件
      videoInfo = getYoutubePlaylistInfo_ydl(urlOrFile)
      isPlayList = True
      try : videos = videoInfo['entries']
      except :
        isPlayList = False
      #處理 Youtube 播放清單
      if isPlayList :
        title = videoInfo['title']
        print('\n>>處理 Youtube 播放清單影片: '+title)
        for video in videos :
          title = video['title'] #取得Youtube影片的標題字
          #filename = title+'.mp3'
          print('\n>>> '+title)
          #用影片的標題當主檔名，但需將不適合的字置換或去除
          #convert title to valid filename
          outputFilename = getFilenameFromTitle(title)

          #如果設定辨識過的重新辨識(overwrite_enable), 或是輸出字幕檔不存在就呼叫whisper
          if not isOutputTextFileExists(outputFilename) or overwrite_enable :
            # delete the old audio file
            removeAudioFile(filename)
            # download audio from video stream
            print('>> 下載影音檔案中...')
            try: getAudioFromYoutube_ydl(video['webpage_url'], tempFolder, filename)
            except:
              print('>>>無法由網址下載影音')

            if os.path.exists(filename) :
              print('>>>Whisper 辨識中...\n')
              # load whisper model and get transcribe result
              transcribe(filename, output_path, outputFilename, outputFormat)
            else :
              print('>>>找不到語音檔，請確定影音檔是否已準備好了\n')

          #將輸出的檔名列入結果清單中，方便後續壓縮、下載
          textFileList.append(getOutputTextFilename(outputFilename))
        title = videoInfo['title']
      else :
        # Youtube single video
        title = videoInfo['title']
        outputFilename = getFilenameFromTitle(title) #用影片的標題當主檔名，但需將不適合的字置換或去除
        #如果設定辨識過的重新辨識(overwrite_enable), 或是輸出字幕檔不存在就呼叫whisper
        if not isOutputTextFileExists(outputFilename) or overwrite_enable :
          # delete the old audio file
          removeAudioFile(filename)
          print('>> 下載影音檔案中...')
          try: getAudioFromYoutube_ydl(videoInfo['webpage_url'], tempFolder, filename)
          except:
            print('>>>無法由網址下載影音')
        #將輸出的檔名列入結果清單中，方便後續壓縮、下載
        textFileList.append(getOutputTextFilename(outputFilename))
    else :
      #非Youtube的網址
      # delete the old audio file
      removeAudioFile(filename)
      isPlayList = False
      if re.search('drive\.google\.com', urlOrFile) :
        #Google drive files
        title = getYoutubeTitle_ydl(urlOrFile) #取得檔名
        outputFilename = getFilenameFromTitle(title) #主檔名，但需將不適合的字置換或去除
        if not isOutputTextFileExists(outputFilename) or overwrite_enable :
          getAudioFromYoutube_ydl(urlOrFile, tempFolder, filename)
        #將輸出的檔名列入結果清單中，方便後續壓縮、下載
        textFileList.append(getOutputTextFilename(outputFilename))
      else :
        # other website       
        title = ''
        removeAudioFile(filename)
        print('\n>> 下載影音檔案中...\n')
        try : videoInfo = getYoutubePlaylistInfo_ydl(urlOrFile)
        except:
          print('\n>>> 無法下載該網址的影音資訊 ...\n')
        try: title = videoInfo['title'] #取得標題字
        except:
          title = 'unknow'
        outputFilename = getFilenameFromTitle(title) #用影片的標題當主檔名，但需將不適合的字置換或去除
        try: 
          getAudioFromYoutube_ydl(videoInfo['webpage_url'], tempFolder, filename)
        except:
          print('\n>>> 無法下載影音\n')
        #將輸出的檔名列入結果清單中，方便後續壓縮、下載
        textFileList.append(getOutputTextFilename(outputFilename))
  else :
    #Colab儲存空間檔案處理(非網址的影音檔)
    isPlayList = False
    title = os.path.splitext(os.path.basename(urlOrFile))[0] #去掉資料夾名稱及附檔名，只取主檔名
    outputFilename = getFilenameFromTitle(title) #結果輸出檔的主檔名
    filename = urlOrFile #辨識指定的影音檔
    if os.path.exists(filename) :
      #將輸出的檔名列入結果清單中，方便後續壓縮、下載
      textFileList.append(getOutputTextFilename(outputFilename))
      #如果字幕檔已存在，而且不可覆蓋，就故意將語音檔設為不存在的檔名，這樣就不會重新辨識
      if (not overwrite_enable) and isOutputTextFileExists(outputFilename) :
        filename = 'fakeFilename.fake'
  #辨識非播放清單的單一語音檔
  if not isPlayList :
    print('\n>> '+title)
    # 語音檔存在的話，就進行辨識
    # load whisper model and get transcribe result
    print('\n>> 準備進行語音辨識...\n')
    if os.path.exists(filename) :
      print('>>>Whisper 辨識中...\n')
      transcribe(filename, output_path, outputFilename, outputFormat)
    else :
      print('>>>找不到語音檔，請確定影音檔是否已準備好了\n')
  return title

#解析 .txt 檔案中的清單，一行行進行去抓音檔並進行辨識
def parseTxtFileAndTranscribe(txtFile) :
  if os.path.exists(txtFile) :
    file = open(txtFile, 'r')
    lines = file.readlines()
    file.close()
    for line in lines :
      line = re.sub('\r|\n', '', line, re.MULTILINE) #去掉所有換行字元
      getAudioAndTranscribe(line)
  else :
    print('找不到檔案: '+txtFile)

if (re.search('https\:\/\/', url) is None) and (not re.search('\.txt$', url, re.IGNORECASE) is None) :
  #解析 .txt 檔中的清單後，再進行辨識
  parseTxtFileAndTranscribe(url)
  title = os.path.splitext(os.path.basename(url))[0] #去掉資料夾名稱及附檔名，只取主檔名
else :
  #直接辨識 url 指定的網址或是檔案
  title = getAudioAndTranscribe(url)
  

if len(textFileList)>0 and start_downloading_immediately :
  print('download...')
  filenames_list = ''
  for textFile in textFileList :
    if os.path.exists(textFile) :
      convertNewLineFormat(textFile) # replace \n with \r\n (Windows format)
      if os.getcwd() != os.path.dirname(os.path.abspath(textFile)) :
        filename = os.path.basename(textFile)
        #copy textFile to current work directory
        ! cp {textFile} ./
      else :
        filename = textFile
      if filenames_list != '' :
        filenames_list = filenames_list+' '
      filenames_list = filenames_list+filename

  if len(textFileList)>1 :
    print('\n壓縮並下載辨識結果')
    outputFilename = slugify(title, allow_unicode=True, lowercase=False)+'-'+outputFormat+'.zip'
    ! zip {outputFilename} -D {filenames_list}
  else :
    print('\n下載辨識結果')
    outputFilename = filenames_list

  google.colab.files.download(outputFilename)